Cargar el histórico y crear features

In [1]:
import os
import pandas as pd
import numpy as np

BASE_DIR = r"C:\AI_Agents_Aire"
DATA_DIR = os.path.join(BASE_DIR, "data")

hist_path = os.path.join(DATA_DIR, "historical_readings.csv")

df = pd.read_csv(hist_path, parse_dates=["timestamp"])
print(df.shape)
df.head()

(182334, 11)


timestamp  district  pm25   pm10  no2  o3     temp_c  \
0 2016-03-12 04:00:00+00:00  HUACHIPA   NaN  109.0  NaN NaN  15.878854   
1 2016-03-12 05:00:00+00:00  HUACHIPA   NaN  113.0  NaN NaN  14.950022   
2 2016-03-12 06:00:00+00:00  HUACHIPA   NaN  101.0  NaN NaN  15.750059   
3 2016-03-12 07:00:00+00:00  HUACHIPA   NaN   98.1  NaN NaN  15.946421   
4 2016-03-12 08:00:00+00:00  HUACHIPA   NaN  131.0  NaN NaN  17.272942   

   humidity_pct   wind_ms        lat        lon  
0     76.603072  2.390308 -12.017094 -76.948597  
1     60.357718  2.327283 -12.017094 -76.948597  
2     76.879161  1.113699 -12.017094 -76.948597  
3     75.410237  2.348253 -12.017094 -76.948597  
4     78.593325  1.421261 -12.017094 -76.948597

Crear columnas de tiempo y limpiar pm10

In [2]:
# Features temporales
df["hour"] = df["timestamp"].dt.hour
df["dayofweek"] = df["timestamp"].dt.dayofweek

# Asegurar que pm10 sea numérico y sin huecos grandes
df["pm10"] = df["pm10"].astype(float)
df["pm10"] = df["pm10"].interpolate().fillna(method="bfill").fillna(method="ffill")

C:\Users\eriveraj\AppData\Local\Temp\ipykernel_30764\1386670269.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["pm10"] = df["pm10"].interpolate().fillna(method="bfill").fillna(method="ffill")


Crear etiqueta de riesgo (Bajo/Medio/Alto) --- Sustentado en la presentacion

In [3]:
def categorize_risk(pm10):
    if pm10 < 50:
        return "Bajo"
    elif pm10 < 100:
        return "Medio"
    else:
        return "Alto"

df["risk_label"] = df["pm10"].apply(categorize_risk)
df["risk_label"].value_counts()


risk_label
Bajo     77949
Medio    62936
Alto     41449
Name: count, dtype: int64

Entrenar modelo de clasificación de riesgo

In [6]:
%pip install scikit-learn

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 538.0 kB/s eta 0:00:00
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.9 MB 2.0 MB/s eta 0:00:05
    --------------------------------------- 0.2/8.9 MB 2.0 MB/s eta 0:00:05
   - -------------------------------------- 0.4/8.9 MB 2.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.5/8.9 MB 3.3 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/8.9 MB 3.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.1/8.9 MB 4.0 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/8.9 MB 3.7 MB/s eta 0:00:03
   ------ ---------------


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\eriveraj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [8]:
# Features que usará el modelo
features = ["pm10", "temp_c", "humidity_pct", "wind_ms", "hour"]
X = df[features]
y = df["risk_label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(
        n_estimators=100,
        random_state=42,
        class_weight="balanced"
    ))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

        Alto       1.00      1.00      1.00      8290
        Bajo       1.00      1.00      1.00     15590
       Medio       1.00      1.00      1.00     12587

    accuracy                           1.00     36467
   macro avg       1.00      1.00      1.00     36467
weighted avg       1.00      1.00      1.00     36467



Elegí Random Forest porque es un modelo robusto que maneja bien relaciones no lineales y datos desbalanceados, y además me permite ver qué variables pesan más a la hora de decidir el riesgo.

Entrené un modelo de clasificación (Random Forest) para predecir el nivel de riesgo Bajo/Medio/Alto.
Como la etiqueta de riesgo la definí directamente a partir de PM10 con una regla clara, el modelo puede aprender esa lógica casi perfecta.
Por eso vemos un 100% de precisión y recall: en la práctica, este modelo está reproduciendo una regla de negocio que yo definí (los umbrales 50 y 100 μg/m³).

En caso de usar Regresion Logistica

from sklearn.linear_model import LogisticRegression

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced", multi_class="auto"))
])

importancia de variables:

In [10]:
clf = pipe.named_steps["clf"]
importancias = pd.Series(clf.feature_importances_, index=features)
importancias.sort_values(ascending=False)

pm10            0.991416
hour            0.005160
temp_c          0.002083
humidity_pct    0.000675
wind_ms         0.000665
dtype: float64

Detección de anomalías

In [12]:
from sklearn.ensemble import IsolationForest
import pandas as pd

iso = IsolationForest(contamination=0.05, random_state=42)
iso.fit(X)

# score de anomalía (mientras más negativo, más raro)
df["anomaly_score"] = iso.decision_function(X)

# predicciones: 1 = normal, -1 = anomalía
y_iso = iso.predict(X)  # esto es un np.ndarray

# lo convertimos a Series y recién ahí usamos map
df["is_anomaly"] = pd.Series(y_iso, index=df.index).map({1: 0, -1: 1})
df["is_anomaly"].value_counts()


is_anomaly
0    173217
1      9117
Name: count, dtype: int64

In [13]:
df["is_anomaly"].value_counts(normalize=True)

is_anomaly
0    0.949998
1    0.050002
Name: proportion, dtype: float64

Entrené un Isolation Forest asumiendo que alrededor del 5% de las mediciones de calidad de aire serían eventos raros o extremos.
El modelo encontró unos 9 mil puntos anómalos sobre un total de ~182 mil registros, es decir, aproximadamente un 5%.
A partir de aquí podemos explorar cuándo y dónde ocurren estas anomalías (por ejemplo, qué distritos o qué horas del día concentran más episodios de contaminación atípica).

Aplicar el modelo y guardar

In [14]:
# Probabilidad de que el riesgo sea Alto
proba = pipe.predict_proba(X)
classes = pipe.named_steps["clf"].classes_
idx_alto = list(classes).index("Alto")
df["risk_score"] = proba[:, idx_alto]

# Etiqueta predicha
df["risk_label_pred"] = pipe.predict(X)

# Dataset con puntuaciones
scored = df[[
    "timestamp", "district", "pm10", "temp_c",
    "humidity_pct", "wind_ms",
    "risk_label_pred", "risk_score", "is_anomaly",
    "lat", "lon"
]].copy()

scored = scored.sort_values("timestamp")

scored_path = os.path.join(DATA_DIR, "scored_readings.csv")
scored.to_csv(scored_path, index=False)
print(f"Archivo con lecturas puntuadas: {scored_path}")
scored.head()


Archivo con lecturas puntuadas: C:\AI_Agents_Aire\data\scored_readings.csv


timestamp  district   pm10     temp_c  humidity_pct  \
0 2016-03-12 04:00:00+00:00  HUACHIPA  109.0  15.878854     76.603072   
1 2016-03-12 05:00:00+00:00  HUACHIPA  113.0  14.950022     60.357718   
2 2016-03-12 06:00:00+00:00  HUACHIPA  101.0  15.750059     76.879161   
3 2016-03-12 07:00:00+00:00  HUACHIPA   98.1  15.946421     75.410237   
4 2016-03-12 08:00:00+00:00  HUACHIPA  131.0  17.272942     78.593325   

    wind_ms risk_label_pred  risk_score  is_anomaly        lat        lon  
0  2.390308            Alto        1.00           0 -12.017094 -76.948597  
1  2.327283            Alto        1.00           0 -12.017094 -76.948597  
2  1.113699            Alto        0.99           0 -12.017094 -76.948597  
3  2.348253           Medio        0.00           0 -12.017094 -76.948597  
4  1.421261            Alto        1.00           0 -12.017094 -76.948597

Crear alerts_summary.csv para Copilot

In [15]:
# Tomemos por ejemplo las últimas 24 horas del histórico
last_ts = scored["timestamp"].max()
cutoff = last_ts - pd.Timedelta(hours=24)
recent = scored[scored["timestamp"] >= cutoff]

summary = (recent
           .groupby("district")
           .agg(
               pm10_max=("pm10", "max"),
               pm10_avg=("pm10", "mean"),
               temp_avg=("temp_c", "mean"),
               hum_avg=("humidity_pct", "mean"),
               wind_avg=("wind_ms", "mean"),
               risk_max=("risk_score", "max"),
               anomalies=("is_anomaly", "sum")
           )
           .reset_index())

def global_risk(score):
    if score < 0.3:
        return "Bajo"
    elif score < 0.6:
        return "Medio"
    else:
        return "Alto"

summary["risk_level"] = summary["risk_max"].apply(global_risk)

summary_path = os.path.join(DATA_DIR, "alerts_summary.csv")
summary.to_csv(summary_path, index=False)
print(f"Resumen para Power Automate: {summary_path}")
summary


Resumen para Power Automate: C:\AI_Agents_Aire\data\alerts_summary.csv


district  pm10_max  pm10_avg   temp_avg    hum_avg  wind_avg  \
0   Huachipa - El Paraiso     126.0    76.448  17.725942  69.289178  1.926702   
1     Huachipa - Nieveria      99.2    49.480  17.929795  70.074505  2.080088   
2  Huachipa - Santa Maria      53.0    28.512  17.904163  70.878143  1.915290   

   risk_max  anomalies risk_level  
0       1.0          0       Alto  
1       0.0          0       Bajo  
2       0.0          0       Bajo